# Reconciliation #06: Tax and NOL Carryforward

## Overview
- **What this notebook tests:** Tax calculations correctly apply the NOL carryforward rules (IRC section 172, 80% limitation), deferred tax assets accumulate and unwind properly, and the tax shield from losses is accurately quantified.
- **Prerequisites:** `pip install -e .` from the project root.
- **Estimated runtime:** < 60 seconds
- **Audience:** Developers, actuaries, and tax accountants verifying ASC 740 / IRC 172 compliance.

## Checks Performed
1. **Profitable Years (Baseline):** Tax = Pre-Tax Income x Tax Rate when no NOL exists
2. **Loss Year (NOL Generation):** Loss creates NOL carryforward; no tax paid; DTA recognized
3. **Recovery Years (NOL Usage):** NOL offsets taxable income subject to 80% limitation per IRC 172(a)(2)
4. **80% Limitation Verification:** NOL deduction never exceeds 80% of current-year taxable income
5. **DTA and Valuation Allowance:** Gross DTA = NOL x Tax Rate; valuation allowance follows ASC 740-10-30-5 thresholds

## Approach
We use `TaxHandler` directly to exercise a controlled sequence of profit and loss years,
verifying NOL accumulation, usage, DTA, and the 80% TCJA limitation at each step.
We also run a manufacturer-level integration test to confirm the full step() pipeline
correctly threads tax/NOL state.

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/reconciliation'

    if os.path.exists(os.path.join(NOTEBOOK_DIR, '_reconciliation_helpers.py')):
        print(f"Found helper module in {NOTEBOOK_DIR}")
        os.chdir(NOTEBOOK_DIR)
        if NOTEBOOK_DIR not in sys.path:
            sys.path.append(NOTEBOOK_DIR)
    else:
        print(f"WARNING: _reconciliation_helpers.py not found in {NOTEBOOK_DIR}")
        print("Please update NOTEBOOK_DIR in this cell to the correct folder path.")

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
"""Setup: imports and notebook header."""
import sys
import os
import warnings
from decimal import Decimal

import numpy as np
import pandas as pd
from IPython.display import display, HTML

# Reconciliation helper utilities
sys.path.insert(0, os.path.dirname(os.path.abspath(".")))
from _reconciliation_helpers import (
    ReconciliationChecker, final_summary, section_header,
    notebook_header, timed_cell, fmt_dollar, display_df,
    create_standard_manufacturer,
)

# Core framework imports
from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.tax_handler import TaxHandler
from ergodic_insurance.accrual_manager import AccrualManager
from ergodic_insurance.ledger import AccountName, Ledger
from ergodic_insurance.decimal_utils import to_decimal, ZERO

# Display the standard notebook header
notebook_header(
    number=6,
    title="Tax and NOL Carryforward",
    description=(
        "Verifies tax calculations with NOL carryforward rules (IRC 172, 80% TCJA limitation), "
        "deferred tax asset accumulation and unwinding, and valuation allowance per ASC 740."
    ),
)

# Suppress verbose logging from the manufacturer during simulation
import logging
logging.getLogger("ergodic_insurance").setLevel(logging.WARNING)

# Fixed parameters for reproducibility
TAX_RATE = 0.25
NOL_LIMIT_PCT = 0.80
TOLERANCE = 0.01  # $0.01 tolerance

print("Setup complete.")
print(f"Tax rate: {TAX_RATE:.0%}")
print(f"NOL limitation: {NOL_LIMIT_PCT:.0%} (IRC 172(a)(2) post-TCJA)")

## Section 1: Setup TaxHandler

We create a standalone `TaxHandler` instance with a 25% tax rate, 80% NOL limitation,
and TCJA rules enabled. This lets us precisely control and verify the NOL lifecycle
without the complexity of a full manufacturer simulation.

In [ ]:
"""Create standalone TaxHandler for controlled NOL testing."""
section_header("1. Setup TaxHandler")

with timed_cell("TaxHandler setup"):
    accrual_mgr = AccrualManager()
    tax_handler = TaxHandler(
        tax_rate=TAX_RATE,
        accrual_manager=accrual_mgr,
        nol_carryforward=Decimal("0"),
        nol_limitation_pct=NOL_LIMIT_PCT,
        apply_tcja_limitation=True,
    )

    print(f"Initial NOL carryforward: {fmt_dollar(tax_handler.nol_carryforward)}")
    print(f"Initial DTA: {fmt_dollar(tax_handler.deferred_tax_asset)}")
    print(f"Initial consecutive loss years: {tax_handler.consecutive_loss_years}")
    print(f"Valuation allowance rate: {float(tax_handler.valuation_allowance_rate):.0%}")

## Section 2: Profitable Years (Baseline)

With no NOL balance, tax on profitable years should be:
```
Tax = Pre-Tax Income x Tax Rate
```
No NOL is consumed, and no DTA is created.

In [ ]:
"""Verify baseline tax calculation on profitable years with no NOL."""
section_header("2. Profitable Years (Baseline)")

checker_baseline = ReconciliationChecker(section="Profitable Years Baseline")

# Define a sequence of profitable years
profitable_incomes = [
    Decimal("1000000"),   # Year 1: $1M
    Decimal("1500000"),   # Year 2: $1.5M
    Decimal("800000"),    # Year 3: $800K
]

baseline_rows = []

with timed_cell("Baseline checks"):
    for i, income in enumerate(profitable_incomes, 1):
        expected_tax = income * to_decimal(TAX_RATE)
        expected_nol_used = Decimal("0")

        # Calculate tax (mutates state)
        actual_tax, nol_used = tax_handler.calculate_tax_liability(income)

        baseline_rows.append({
            "Year": i,
            "Pre-Tax Income": float(income),
            "NOL Balance (before)": 0.0,
            "NOL Used": float(nol_used),
            "Tax Paid": float(actual_tax),
            "Expected Tax": float(expected_tax),
            "DTA": float(tax_handler.deferred_tax_asset),
        })

        checker_baseline.assert_close(
            float(actual_tax), float(expected_tax), tol=TOLERANCE,
            message=f"Year {i}: Tax = Income x Rate",
            label_actual="Actual Tax", label_expected="Expected Tax",
        )
        checker_baseline.assert_close(
            float(nol_used), 0.0, tol=TOLERANCE,
            message=f"Year {i}: No NOL consumed",
            label_actual="NOL Used", label_expected="0",
        )
        checker_baseline.assert_close(
            float(tax_handler.nol_carryforward), 0.0, tol=TOLERANCE,
            message=f"Year {i}: NOL balance remains zero",
            label_actual="NOL Balance", label_expected="0",
        )
        checker_baseline.assert_close(
            float(tax_handler.deferred_tax_asset), 0.0, tol=TOLERANCE,
            message=f"Year {i}: DTA remains zero",
            label_actual="DTA", label_expected="0",
        )

df_baseline = pd.DataFrame(baseline_rows)
for col in ["Pre-Tax Income", "NOL Used", "Tax Paid", "Expected Tax", "DTA"]:
    df_baseline[col] = df_baseline[col].map(lambda x: f"${x:,.2f}")
display_df(df_baseline, title="Profitable Years -- Tax Calculation")

checker_baseline.display_results()

## Section 3: Loss Year (NOL Generation)

When a company incurs a loss:
- Tax = $0 (no tax on negative income)
- The loss is accumulated into `nol_carryforward`
- DTA = NOL Balance x Tax Rate (ASC 740-10-25-3)
- `consecutive_loss_years` increments

In [ ]:
"""Verify NOL generation from a loss year."""
section_header("3. Loss Year (NOL Generation)")

checker_loss = ReconciliationChecker(section="Loss Year NOL Generation")

with timed_cell("Loss year checks"):
    # Year 4: Large loss
    loss_amount = Decimal("-2000000")  # $2M loss
    nol_before = tax_handler.nol_carryforward
    consecutive_before = tax_handler.consecutive_loss_years

    actual_tax, nol_used = tax_handler.calculate_tax_liability(loss_amount)

    # Expected results
    expected_nol_after = nol_before + abs(loss_amount)  # $2M NOL
    expected_dta = expected_nol_after * to_decimal(TAX_RATE)  # $500K DTA

    print(f"Loss amount: {fmt_dollar(loss_amount)}")
    print(f"NOL before: {fmt_dollar(nol_before)}")
    print(f"NOL after: {fmt_dollar(tax_handler.nol_carryforward)}")
    print(f"DTA: {fmt_dollar(tax_handler.deferred_tax_asset)}")
    print(f"Consecutive loss years: {tax_handler.consecutive_loss_years}")

    # Check: no tax on loss
    checker_loss.assert_close(
        float(actual_tax), 0.0, tol=TOLERANCE,
        message="Loss year: Tax = $0",
        label_actual="Tax", label_expected="0",
    )

    # Check: no NOL consumed on loss year
    checker_loss.assert_close(
        float(nol_used), 0.0, tol=TOLERANCE,
        message="Loss year: No NOL consumed",
        label_actual="NOL Used", label_expected="0",
    )

    # Check: NOL balance accumulated
    checker_loss.assert_close(
        float(tax_handler.nol_carryforward), float(expected_nol_after), tol=TOLERANCE,
        message="Loss year: NOL balance = prior NOL + |loss|",
        label_actual="NOL Balance", label_expected="Expected NOL",
    )

    # Check: DTA = NOL x Tax Rate
    checker_loss.assert_close(
        float(tax_handler.deferred_tax_asset), float(expected_dta), tol=TOLERANCE,
        message="Loss year: DTA = NOL x Tax Rate",
        label_actual="DTA", label_expected="Expected DTA",
    )

    # Check: consecutive loss years incremented
    checker_loss.assert_equal(
        tax_handler.consecutive_loss_years, consecutive_before + 1,
        message="Loss year: consecutive_loss_years incremented",
    )

checker_loss.display_results()

## Section 4: Recovery Years (NOL Usage)

When a company returns to profitability with NOL carryforward available:
- NOL deduction is limited to 80% of current-year taxable income (IRC 172(a)(2) post-TCJA)
- Tax = max(0, (Taxable Income - NOL Used) x Tax Rate)
- NOL balance decreases by the amount used
- DTA decreases proportionally
- `consecutive_loss_years` resets to 0

We test two recovery years to show partial and full NOL consumption.

In [ ]:
"""Verify NOL usage during recovery years with 80% limitation."""
section_header("4. Recovery Years (NOL Usage)")

checker_recovery = ReconciliationChecker(section="Recovery Years NOL Usage")

# Current state: NOL = $2M from the loss year, consecutive_loss_years = 1
recovery_incomes = [
    Decimal("1500000"),   # Year 5: $1.5M profit -- 80% limit = $1.2M NOL used
    Decimal("1500000"),   # Year 6: $1.5M profit -- remaining $800K NOL, 80% limit = $1.2M, use $800K
]

recovery_rows = []

with timed_cell("Recovery year checks"):
    for i, income in enumerate(recovery_incomes, 5):
        nol_before = tax_handler.nol_carryforward
        dta_before = tax_handler.deferred_tax_asset

        # Calculate expected values BEFORE the call
        max_nol_deduction = income * to_decimal(NOL_LIMIT_PCT)  # 80% of income
        expected_nol_used = min(nol_before, max_nol_deduction)
        expected_taxable_income = income - expected_nol_used
        expected_tax = max(Decimal("0"), expected_taxable_income * to_decimal(TAX_RATE))
        expected_nol_after = nol_before - expected_nol_used
        expected_dta_after = expected_nol_after * to_decimal(TAX_RATE)

        # Calculate tax (mutates state)
        actual_tax, nol_used = tax_handler.calculate_tax_liability(income)

        recovery_rows.append({
            "Year": i,
            "Pre-Tax Income": float(income),
            "NOL Balance (before)": float(nol_before),
            "80% Limit": float(max_nol_deduction),
            "NOL Used": float(nol_used),
            "Taxable Income": float(income - nol_used),
            "Tax Paid": float(actual_tax),
            "NOL Balance (after)": float(tax_handler.nol_carryforward),
            "DTA (after)": float(tax_handler.deferred_tax_asset),
        })

        # Check: tax amount
        checker_recovery.assert_close(
            float(actual_tax), float(expected_tax), tol=TOLERANCE,
            message=f"Year {i}: Tax = (Income - NOL Used) x Rate",
            label_actual="Actual Tax", label_expected="Expected Tax",
        )

        # Check: NOL used
        checker_recovery.assert_close(
            float(nol_used), float(expected_nol_used), tol=TOLERANCE,
            message=f"Year {i}: NOL used = min(NOL balance, 80% of income)",
            label_actual="NOL Used", label_expected="Expected NOL Used",
        )

        # Check: NOL balance after
        checker_recovery.assert_close(
            float(tax_handler.nol_carryforward), float(expected_nol_after), tol=TOLERANCE,
            message=f"Year {i}: NOL balance decreased by NOL used",
            label_actual="NOL After", label_expected="Expected NOL After",
        )

        # Check: DTA after
        checker_recovery.assert_close(
            float(tax_handler.deferred_tax_asset), float(expected_dta_after), tol=TOLERANCE,
            message=f"Year {i}: DTA = remaining NOL x Tax Rate",
            label_actual="DTA After", label_expected="Expected DTA After",
        )

        # Check: consecutive loss years reset on profit
        checker_recovery.assert_equal(
            tax_handler.consecutive_loss_years, 0,
            message=f"Year {i}: consecutive_loss_years reset to 0",
        )

df_recovery = pd.DataFrame(recovery_rows)
for col in ["Pre-Tax Income", "NOL Balance (before)", "80% Limit", "NOL Used",
            "Taxable Income", "Tax Paid", "NOL Balance (after)", "DTA (after)"]:
    df_recovery[col] = df_recovery[col].map(lambda x: f"${x:,.2f}")
display_df(df_recovery, title="Recovery Years -- NOL Usage with 80% Limitation")

# Verify NOL is fully consumed after year 6
checker_recovery.assert_close(
    float(tax_handler.nol_carryforward), 0.0, tol=TOLERANCE,
    message="After recovery: NOL fully consumed",
    label_actual="Remaining NOL", label_expected="0",
)

checker_recovery.display_results()

## Section 5: 80% Limitation Verification

This section explicitly verifies the 80% TCJA limitation by creating a scenario
where the NOL is larger than current-year income, confirming the deduction is
capped at exactly 80% of taxable income. We also test the pre-TCJA case (100%
deduction) for contrast.

In [ ]:
"""Verify 80% limitation explicitly with large NOL vs small income."""
section_header("5. 80% Limitation Verification")

checker_80pct = ReconciliationChecker(section="80% Limitation")

with timed_cell("80% limit checks"):
    # --- Post-TCJA: 80% limitation ---
    am_tcja = AccrualManager()
    th_tcja = TaxHandler(
        tax_rate=TAX_RATE,
        accrual_manager=am_tcja,
        nol_carryforward=Decimal("5000000"),  # $5M NOL
        nol_limitation_pct=NOL_LIMIT_PCT,
        apply_tcja_limitation=True,
    )

    income_tcja = Decimal("1000000")  # $1M income
    max_deduction_tcja = income_tcja * to_decimal(NOL_LIMIT_PCT)  # $800K
    expected_nol_used_tcja = min(th_tcja.nol_carryforward, max_deduction_tcja)  # $800K
    expected_taxable_tcja = income_tcja - expected_nol_used_tcja  # $200K
    expected_tax_tcja = expected_taxable_tcja * to_decimal(TAX_RATE)  # $50K

    tax_tcja, nol_used_tcja = th_tcja.calculate_tax_liability(income_tcja)

    print("Post-TCJA (80% limitation):")
    print(f"  Income: {fmt_dollar(income_tcja)}")
    print(f"  NOL available: $5,000,000")
    print(f"  80% limit: {fmt_dollar(max_deduction_tcja)}")
    print(f"  NOL used: {fmt_dollar(nol_used_tcja)}")
    print(f"  Tax paid: {fmt_dollar(tax_tcja)}")
    print(f"  Remaining NOL: {fmt_dollar(th_tcja.nol_carryforward)}")

    # Post-TCJA checks
    checker_80pct.assert_close(
        float(nol_used_tcja), float(expected_nol_used_tcja), tol=TOLERANCE,
        message="Post-TCJA: NOL used = 80% of income (not full NOL)",
        label_actual="NOL Used", label_expected="80% of Income",
    )
    checker_80pct.assert_close(
        float(tax_tcja), float(expected_tax_tcja), tol=TOLERANCE,
        message="Post-TCJA: Tax on remaining 20% of income",
        label_actual="Tax Paid", label_expected="Expected Tax",
    )
    checker_80pct.assert_close(
        float(th_tcja.nol_carryforward), float(Decimal("5000000") - expected_nol_used_tcja),
        tol=TOLERANCE,
        message="Post-TCJA: NOL balance reduced by amount used",
        label_actual="Remaining NOL", label_expected="Expected NOL",
    )

    # --- Pre-TCJA: 100% limitation ---
    am_pre = AccrualManager()
    th_pre = TaxHandler(
        tax_rate=TAX_RATE,
        accrual_manager=am_pre,
        nol_carryforward=Decimal("5000000"),  # $5M NOL
        nol_limitation_pct=1.0,
        apply_tcja_limitation=False,  # Pre-TCJA: no percentage limitation
    )

    income_pre = Decimal("1000000")
    tax_pre, nol_used_pre = th_pre.calculate_tax_liability(income_pre)

    print("\nPre-TCJA (100% deduction):")
    print(f"  Income: {fmt_dollar(income_pre)}")
    print(f"  NOL used: {fmt_dollar(nol_used_pre)}")
    print(f"  Tax paid: {fmt_dollar(tax_pre)}")
    print(f"  Remaining NOL: {fmt_dollar(th_pre.nol_carryforward)}")

    # Pre-TCJA checks
    checker_80pct.assert_close(
        float(nol_used_pre), float(income_pre), tol=TOLERANCE,
        message="Pre-TCJA: NOL used = 100% of income",
        label_actual="NOL Used", label_expected="Full Income",
    )
    checker_80pct.assert_close(
        float(tax_pre), 0.0, tol=TOLERANCE,
        message="Pre-TCJA: Tax = $0 (full NOL offset)",
        label_actual="Tax Paid", label_expected="0",
    )
    checker_80pct.assert_close(
        float(th_pre.nol_carryforward), 4_000_000.0, tol=TOLERANCE,
        message="Pre-TCJA: NOL reduced by full income amount",
        label_actual="Remaining NOL", label_expected="$4,000,000",
    )

    # Tax savings comparison
    tax_no_nol = income_tcja * to_decimal(TAX_RATE)
    tcja_savings = tax_no_nol - tax_tcja
    pre_tcja_savings = tax_no_nol - tax_pre
    print(f"\nTax savings comparison (vs. no NOL):")
    print(f"  Post-TCJA savings: {fmt_dollar(tcja_savings)} ({float(tcja_savings/tax_no_nol):.0%})")
    print(f"  Pre-TCJA savings: {fmt_dollar(pre_tcja_savings)} ({float(pre_tcja_savings/tax_no_nol):.0%})")

    checker_80pct.assert_greater(
        float(pre_tcja_savings), float(tcja_savings),
        message="Pre-TCJA saves more than post-TCJA (no 80% cap)",
    )

checker_80pct.display_results()

## Section 6: DTA and Valuation Allowance

Per ASC 740-10-30-5, a valuation allowance reduces the DTA when realization is
not "more likely than not". The allowance schedule is:
- < 3 consecutive loss years: 0% allowance
- 3 consecutive loss years: 50% allowance
- 4 consecutive loss years: 75% allowance
- 5+ consecutive loss years: 100% allowance

We verify:
- Gross DTA = NOL x Tax Rate at each step
- Valuation allowance applies correctly based on consecutive losses
- Net DTA = Gross DTA - Valuation Allowance
- Allowance reverses when profitability returns

In [ ]:
"""Verify DTA calculation and valuation allowance lifecycle."""
section_header("6. DTA and Valuation Allowance")

checker_dta = ReconciliationChecker(section="DTA and Valuation Allowance")

with timed_cell("DTA/VA checks"):
    # Create a fresh handler for the valuation allowance lifecycle test
    am_va = AccrualManager()
    th_va = TaxHandler(
        tax_rate=TAX_RATE,
        accrual_manager=am_va,
        nol_carryforward=Decimal("0"),
        nol_limitation_pct=NOL_LIMIT_PCT,
        apply_tcja_limitation=True,
    )

    # Scenario: 5 consecutive loss years followed by a profitable year
    # This exercises the full valuation allowance schedule
    scenario = [
        ("Loss 1", Decimal("-500000")),    # Year 1: 1 loss year -> 0% VA
        ("Loss 2", Decimal("-500000")),    # Year 2: 2 loss years -> 0% VA
        ("Loss 3", Decimal("-500000")),    # Year 3: 3 loss years -> 50% VA
        ("Loss 4", Decimal("-500000")),    # Year 4: 4 loss years -> 75% VA
        ("Loss 5", Decimal("-500000")),    # Year 5: 5 loss years -> 100% VA
        ("Profit", Decimal("3000000")),     # Year 6: profit -> 0% VA (reset)
    ]

    expected_va_rates = [
        Decimal("0"),     # 1 loss year
        Decimal("0"),     # 2 loss years
        Decimal("0.50"),  # 3 loss years
        Decimal("0.75"),  # 4 loss years
        Decimal("1.00"),  # 5 loss years
        Decimal("0"),     # profit year (reset)
    ]

    va_rows = []
    cumulative_nol = Decimal("0")

    for idx, ((label, income), expected_rate) in enumerate(zip(scenario, expected_va_rates)):
        nol_before = th_va.nol_carryforward

        tax, nol_used = th_va.calculate_tax_liability(income)

        gross_dta = th_va.deferred_tax_asset
        va_rate = th_va.valuation_allowance_rate
        va_amount = th_va.valuation_allowance
        net_dta = th_va.net_deferred_tax_asset

        # Expected gross DTA
        expected_gross_dta = th_va.nol_carryforward * to_decimal(TAX_RATE)
        expected_va_amount = expected_gross_dta * expected_rate
        expected_net_dta = expected_gross_dta - expected_va_amount

        va_rows.append({
            "Year": label,
            "Income": float(income),
            "NOL Balance": float(th_va.nol_carryforward),
            "Consec Losses": th_va.consecutive_loss_years,
            "Gross DTA": float(gross_dta),
            "VA Rate": f"{float(va_rate):.0%}",
            "VA Amount": float(va_amount),
            "Net DTA": float(net_dta),
        })

        # Check: Gross DTA = NOL x Tax Rate
        checker_dta.assert_close(
            float(gross_dta), float(expected_gross_dta), tol=TOLERANCE,
            message=f"{label}: Gross DTA = NOL x Tax Rate",
            label_actual="Gross DTA", label_expected="Expected",
        )

        # Check: VA rate matches expected schedule
        checker_dta.assert_close(
            float(va_rate), float(expected_rate), tol=TOLERANCE,
            message=f"{label}: VA rate = {float(expected_rate):.0%}",
            label_actual="VA Rate", label_expected="Expected Rate",
        )

        # Check: Net DTA = Gross DTA - VA
        checker_dta.assert_close(
            float(net_dta), float(expected_net_dta), tol=TOLERANCE,
            message=f"{label}: Net DTA = Gross DTA - VA",
            label_actual="Net DTA", label_expected="Expected Net DTA",
        )

df_va = pd.DataFrame(va_rows)
for col in ["Income", "NOL Balance", "Gross DTA", "VA Amount", "Net DTA"]:
    df_va[col] = df_va[col].map(lambda x: f"${x:,.2f}")
display_df(df_va, title="DTA and Valuation Allowance Lifecycle")

# After the profit year, VA should be reversed
checker_dta.assert_close(
    float(th_va.valuation_allowance_rate), 0.0, tol=TOLERANCE,
    message="After profit: VA rate reverts to 0%",
    label_actual="VA Rate", label_expected="0%",
)
checker_dta.assert_equal(
    th_va.consecutive_loss_years, 0,
    message="After profit: consecutive_loss_years = 0",
)

checker_dta.display_results()

## Section 7: Year-by-Year Tracking Table

A comprehensive end-to-end test running through a complete NOL lifecycle using
a fresh TaxHandler. This combines all checks into a single consolidated table:
Income | NOL Balance | NOL Used | Tax Paid | DTA.

In [ ]:
"""Complete year-by-year NOL lifecycle tracking table."""
section_header("7. Complete Year-by-Year Lifecycle")

checker_lifecycle = ReconciliationChecker(section="Full Lifecycle")

with timed_cell("Lifecycle table"):
    am_full = AccrualManager()
    th_full = TaxHandler(
        tax_rate=TAX_RATE,
        accrual_manager=am_full,
        nol_carryforward=Decimal("0"),
        nol_limitation_pct=NOL_LIMIT_PCT,
        apply_tcja_limitation=True,
    )

    # Full lifecycle: profit -> loss -> recovery -> profit
    incomes = [
        ("Year 1", Decimal("1200000")),     # Profitable
        ("Year 2", Decimal("1000000")),     # Profitable
        ("Year 3", Decimal("-3000000")),    # Large loss -> NOL = $3M
        ("Year 4", Decimal("-500000")),     # Another loss -> NOL = $3.5M
        ("Year 5", Decimal("2000000")),     # Recovery: 80% limit = $1.6M NOL used
        ("Year 6", Decimal("2000000")),     # Recovery: 80% limit = $1.6M, NOL = $1.9M, use $1.6M
        ("Year 7", Decimal("500000")),      # Recovery: 80% limit = $400K, NOL = $300K, use $300K
        ("Year 8", Decimal("1000000")),     # Fully recovered: no NOL left
    ]

    lifecycle_rows = []

    for label, income in incomes:
        nol_before = th_full.nol_carryforward
        dta_before = th_full.deferred_tax_asset

        # Compute expected results
        if income <= ZERO:
            exp_nol_used = Decimal("0")
            exp_tax = Decimal("0")
            exp_nol_after = nol_before + abs(income)
        else:
            max_deduction = income * to_decimal(NOL_LIMIT_PCT)
            exp_nol_used = min(nol_before, max_deduction)
            exp_tax = max(Decimal("0"), (income - exp_nol_used) * to_decimal(TAX_RATE))
            exp_nol_after = nol_before - exp_nol_used
        exp_dta_after = exp_nol_after * to_decimal(TAX_RATE)

        # Execute
        actual_tax, actual_nol_used = th_full.calculate_tax_liability(income)

        lifecycle_rows.append({
            "Period": label,
            "Income": float(income),
            "NOL (before)": float(nol_before),
            "NOL Used": float(actual_nol_used),
            "Tax Paid": float(actual_tax),
            "NOL (after)": float(th_full.nol_carryforward),
            "DTA": float(th_full.deferred_tax_asset),
        })

        # Verify
        checker_lifecycle.assert_close(
            float(actual_tax), float(exp_tax), tol=TOLERANCE,
            message=f"{label}: Tax amount correct",
            label_actual="Tax", label_expected="Expected",
        )
        checker_lifecycle.assert_close(
            float(actual_nol_used), float(exp_nol_used), tol=TOLERANCE,
            message=f"{label}: NOL usage correct",
            label_actual="NOL Used", label_expected="Expected",
        )
        checker_lifecycle.assert_close(
            float(th_full.nol_carryforward), float(exp_nol_after), tol=TOLERANCE,
            message=f"{label}: NOL balance correct",
            label_actual="NOL After", label_expected="Expected",
        )
        checker_lifecycle.assert_close(
            float(th_full.deferred_tax_asset), float(exp_dta_after), tol=TOLERANCE,
            message=f"{label}: DTA correct",
            label_actual="DTA", label_expected="Expected",
        )

df_lifecycle = pd.DataFrame(lifecycle_rows)
for col in ["Income", "NOL (before)", "NOL Used", "Tax Paid", "NOL (after)", "DTA"]:
    df_lifecycle[col] = df_lifecycle[col].map(lambda x: f"${x:,.2f}")
display_df(df_lifecycle, title="Complete Year-by-Year NOL Lifecycle")

# Final state checks
checker_lifecycle.assert_close(
    float(th_full.nol_carryforward), 0.0, tol=TOLERANCE,
    message="End: All NOL consumed",
    label_actual="Final NOL", label_expected="0",
)
checker_lifecycle.assert_close(
    float(th_full.deferred_tax_asset), 0.0, tol=TOLERANCE,
    message="End: DTA = 0",
    label_actual="Final DTA", label_expected="0",
)

checker_lifecycle.display_results()

## Section 8: Manufacturer Integration Test

Verify that the full manufacturer `step()` pipeline correctly threads tax/NOL
state through the income statement, balance sheet, and DTA ledger entries.
We create a manufacturer with NOL enabled, induce a loss year by recording a
large uninsured claim, and confirm the DTA appears in subsequent years.

In [ ]:
"""Integration test: manufacturer step() with NOL threading."""
section_header("8. Manufacturer Integration Test")

checker_mfr = ReconciliationChecker(section="Manufacturer Integration")

with timed_cell("Manufacturer integration"):
    # Create manufacturer with NOL enabled and higher margin to stay solvent.
    # We use a 15% margin so operating income ~$1.5M, then record a large
    # insurance loss via period_insurance_losses to create a pre-tax loss
    # without draining cash (which could cause insolvency).
    config = ManufacturerConfig(
        initial_assets=10_000_000,
        asset_turnover_ratio=1.0,
        base_operating_margin=0.15,
        tax_rate=TAX_RATE,
        retention_ratio=0.70,
        nol_carryforward_enabled=True,
        nol_limitation_pct=NOL_LIMIT_PCT,
        apply_tcja_limitation=True,
        lae_ratio=0.0,
    )
    mfr = WidgetManufacturer(config)

    # Year 1: Normal profitable year (baseline)
    metrics_y1 = mfr.step(
        letter_of_credit_rate=0.015,
        growth_rate=0.0,
        time_resolution="annual",
        apply_stochastic=False,
    )
    nol_y1 = float(mfr.tax_handler.nol_carryforward)
    dta_y1 = float(mfr.tax_handler.deferred_tax_asset)

    checker_mfr.assert_close(
        nol_y1, 0.0, tol=TOLERANCE,
        message="Year 1 (profitable): NOL = 0",
        label_actual="NOL", label_expected="0",
    )
    checker_mfr.assert_close(
        dta_y1, 0.0, tol=TOLERANCE,
        message="Year 1 (profitable): DTA = 0",
        label_actual="DTA", label_expected="0",
    )
    print(f"Year 1: Net income = {fmt_dollar(metrics_y1['net_income'])}")
    print(f"  NOL = {fmt_dollar(nol_y1)}, DTA = {fmt_dollar(dta_y1)}")

    # Year 2: Induce a loss by recording insurance losses
    # Operating income ~$1.5M, so $2M loss -> ~$500K pre-tax loss -> NOL created.
    # We set period_insurance_losses directly. This is the mechanism used when
    # claims exceed insurance limits and the company absorbs the loss.
    mfr.period_insurance_losses = to_decimal(2_000_000)
    metrics_y2 = mfr.step(
        letter_of_credit_rate=0.015,
        growth_rate=0.0,
        time_resolution="annual",
        apply_stochastic=False,
    )
    nol_y2 = float(mfr.tax_handler.nol_carryforward)
    dta_y2 = float(mfr.tax_handler.deferred_tax_asset)

    checker_mfr.check(
        float(metrics_y2["net_income"]) < 0,
        message="Year 2 (loss): Net income is negative",
        detail=f"Net income = {fmt_dollar(metrics_y2['net_income'])}",
    )
    checker_mfr.check(
        nol_y2 > 0,
        message="Year 2 (loss): NOL carryforward created",
        detail=f"NOL = {fmt_dollar(nol_y2)}",
    )
    checker_mfr.assert_close(
        dta_y2, nol_y2 * TAX_RATE, tol=TOLERANCE,
        message="Year 2 (loss): DTA = NOL x Tax Rate",
        label_actual="DTA", label_expected="NOL x Rate",
    )

    # Check DTA is recorded on the ledger
    ledger_dta = float(mfr.ledger.get_balance(AccountName.DEFERRED_TAX_ASSET))
    checker_mfr.assert_close(
        ledger_dta, dta_y2, tol=TOLERANCE,
        message="Year 2 (loss): DTA on ledger matches TaxHandler",
        label_actual="Ledger DTA", label_expected="TaxHandler DTA",
    )
    print(f"\nYear 2: Net income = {fmt_dollar(metrics_y2['net_income'])}")
    print(f"  NOL = {fmt_dollar(nol_y2)}, DTA = {fmt_dollar(dta_y2)}")
    print(f"  Ledger DTA = {fmt_dollar(ledger_dta)}")

    # Year 3: Recovery year - NOL should be partially consumed
    metrics_y3 = mfr.step(
        letter_of_credit_rate=0.015,
        growth_rate=0.0,
        time_resolution="annual",
        apply_stochastic=False,
    )
    nol_y3 = float(mfr.tax_handler.nol_carryforward)
    dta_y3 = float(mfr.tax_handler.deferred_tax_asset)

    checker_mfr.check(
        nol_y3 < nol_y2,
        message="Year 3 (recovery): NOL decreased",
        detail=f"NOL: {fmt_dollar(nol_y2)} -> {fmt_dollar(nol_y3)}",
    )
    checker_mfr.check(
        dta_y3 < dta_y2,
        message="Year 3 (recovery): DTA decreased",
        detail=f"DTA: {fmt_dollar(dta_y2)} -> {fmt_dollar(dta_y3)}",
    )
    checker_mfr.assert_close(
        dta_y3, nol_y3 * TAX_RATE, tol=TOLERANCE,
        message="Year 3 (recovery): DTA = NOL x Tax Rate",
        label_actual="DTA", label_expected="NOL x Rate",
    )
    print(f"\nYear 3: Net income = {fmt_dollar(metrics_y3['net_income'])}")
    print(f"  NOL = {fmt_dollar(nol_y3)}, DTA = {fmt_dollar(dta_y3)}")

    # Verify consecutive loss years reset on recovery
    checker_mfr.assert_equal(
        mfr.tax_handler.consecutive_loss_years, 0,
        message="Year 3 (recovery): consecutive_loss_years = 0",
    )

checker_mfr.display_results()

## Section 9: estimate_tax_liability Purity Check

The `estimate_tax_liability()` method is a pure query that must NOT mutate
NOL state. This is critical for metrics calculations and the decision engine
(Issue #1331). We verify calling it repeatedly returns identical results and
leaves the TaxHandler state unchanged.

In [ ]:
"""Verify estimate_tax_liability is side-effect-free."""
section_header("9. estimate_tax_liability Purity")

checker_purity = ReconciliationChecker(section="Estimate Purity")

with timed_cell("Purity checks"):
    am_pure = AccrualManager()
    th_pure = TaxHandler(
        tax_rate=TAX_RATE,
        accrual_manager=am_pure,
        nol_carryforward=Decimal("1000000"),  # $1M NOL
        nol_limitation_pct=NOL_LIMIT_PCT,
        apply_tcja_limitation=True,
        consecutive_loss_years=2,
    )

    original_nol = th_pure.nol_carryforward
    original_consec = th_pure.consecutive_loss_years

    # Call estimate 5 times with profit
    results = []
    for _ in range(5):
        tax, nol_used = th_pure.estimate_tax_liability(Decimal("500000"))
        results.append((float(tax), float(nol_used)))

    # All results should be identical
    checker_purity.check(
        all(r == results[0] for r in results),
        message="Profit: 5 calls return identical results",
        detail=f"Result: tax={results[0][0]:,.2f}, nol_used={results[0][1]:,.2f}",
    )
    checker_purity.assert_close(
        float(th_pure.nol_carryforward), float(original_nol), tol=TOLERANCE,
        message="Profit: NOL unchanged after estimate calls",
        label_actual="NOL After", label_expected="Original NOL",
    )
    checker_purity.assert_equal(
        th_pure.consecutive_loss_years, original_consec,
        message="Profit: consecutive_loss_years unchanged",
    )

    # Call estimate with loss
    for _ in range(5):
        tax_loss, nol_loss = th_pure.estimate_tax_liability(Decimal("-500000"))

    checker_purity.assert_close(
        float(th_pure.nol_carryforward), float(original_nol), tol=TOLERANCE,
        message="Loss: NOL unchanged after estimate calls",
        label_actual="NOL After", label_expected="Original NOL",
    )
    checker_purity.assert_equal(
        th_pure.consecutive_loss_years, original_consec,
        message="Loss: consecutive_loss_years unchanged",
    )

    # Verify estimate matches calculate for the same inputs
    est_tax, est_nol = th_pure.estimate_tax_liability(Decimal("500000"))
    # Use a clone for calculate to avoid mutating th_pure
    am_clone = AccrualManager()
    th_clone = TaxHandler(
        tax_rate=TAX_RATE,
        accrual_manager=am_clone,
        nol_carryforward=Decimal("1000000"),
        nol_limitation_pct=NOL_LIMIT_PCT,
        apply_tcja_limitation=True,
    )
    calc_tax, calc_nol = th_clone.calculate_tax_liability(Decimal("500000"))

    checker_purity.assert_close(
        float(est_tax), float(calc_tax), tol=TOLERANCE,
        message="Estimate and calculate return same tax",
        label_actual="Estimate", label_expected="Calculate",
    )
    checker_purity.assert_close(
        float(est_nol), float(calc_nol), tol=TOLERANCE,
        message="Estimate and calculate return same NOL used",
        label_actual="Estimate", label_expected="Calculate",
    )

checker_purity.display_results()

## Section 10: Combined Summary

A consolidated view of all checks across all test sections.

In [ ]:
"""Combined summary of all checks."""
section_header("10. Combined Summary")

sections = [
    ("Profitable Years (Baseline)", checker_baseline),
    ("Loss Year (NOL Generation)", checker_loss),
    ("Recovery Years (NOL Usage)", checker_recovery),
    ("80% Limitation Verification", checker_80pct),
    ("DTA and Valuation Allowance", checker_dta),
    ("Full Lifecycle Table", checker_lifecycle),
    ("Manufacturer Integration", checker_mfr),
    ("Estimate Purity", checker_purity),
]

summary_rows = []
for name, checker in sections:
    passed, failed = checker.summary_counts
    total = passed + failed
    summary_rows.append({
        "Check Section": name,
        "Passed": passed,
        "Failed": failed,
        "Total": total,
        "Status": "ALL PASS" if failed == 0 else f"{failed} FAILED",
    })

df_summary = pd.DataFrame(summary_rows)
display_df(df_summary, title="Reconciliation Check Summary")

total_passed = sum(r["Passed"] for r in summary_rows)
total_failed = sum(r["Failed"] for r in summary_rows)
total_checks = total_passed + total_failed
print(f"\nTotal: {total_passed}/{total_checks} checks passed")
if total_failed == 0:
    print("All tax and NOL carryforward checks PASSED.")
else:
    print(f"WARNING: {total_failed} check(s) FAILED.")

## Final Result

The cell below produces the final PASS/FAIL banner. If any check failed,
it raises an `AssertionError` so that `nbconvert` or CI will catch the failure.

In [ ]:
"""Final PASS/FAIL summary -- raises AssertionError on failure."""
final_summary(
    checker_baseline,
    checker_loss,
    checker_recovery,
    checker_80pct,
    checker_dta,
    checker_lifecycle,
    checker_mfr,
    checker_purity,
)